## Final Project Submission

Please fill out:
* Student name: Joseph Husney
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


In [1]:
# import modules needed for project
import requests
import pandas as pd
from tqdm import tqdm

In [3]:
# Get all movie id's before 2019
df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
print(len(df))
print(df.columns)
movie_ids = []
for index, rows in df.iterrows():
    if int(df['release_date'][0].split('-')[0]) < 2019:
        my_list = [rows.id]
        movie_ids.append(my_list)
print(movie_ids)

26517
Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')


[[12444], [10191], [10138], [862], [27205], [32657], [19995], [10193], [20352], [38055], [863], [12155], [27578], [38757], [41439], [24021], [20504], [52371], [2502], [11324], [2501], [37834], [10140], [41233], [34544], [45269], [9737], [10196], [34851], [37799], [38234], [568], [95], [2503], [18823], [34806], [20662], [35019], [10192], [37735], [27022], [22538], [38365], [239], [41283], [7978], [44048], [44214], [44683], [61979], [23759], [27581], [23168], [20533], [20526], [9543], [9994], [7980], [705], [44264], [43947], [23483], [51888], [27576], [47626], [26389], [40662], [42949], [23023], [23631], [25376], [35056], [41733], [31867], [43641], [28874], [38575], [33217], [29427], [33613], [39451], [39514], [22971], [22894], [37933], [25195], [38408], [24253], [38363], [44244], [22972], [30061], [23169], [44115], [47533], [43593], [37861], [12201], [35552], [32856], [43539], [50393], [26388], [11645], [41135], [38745], [41216], [23172], [16290], [13078], [15357], [10664], [51170], [38

In [4]:
# Getting so far info about 1000 movies based on 1000 id's that I got from dataset I was given
# There are 26 thousand movies but it takes very long
movie_info = []
for id_ in movie_ids[:1000]:
    url = f"https://api.themoviedb.org/3/movie/{id_[0]}?api_key=34017996022f64249a86491295577f6b&language=en-US"
    response = requests.get(url)
    movie_info.append(response.json())
df = pd.DataFrame.from_dict(movie_info)
df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,status_code,status_message
0,False,/fXRXpzxUApE3OuXhIqsdavQjCVa.jpg,"{'id': 1241, 'name': 'Harry Potter Collection'...",250000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",https://www.warnerbros.com/movies/harry-potter...,12444.0,tt0926084,en,Harry Potter and the Deathly Hallows: Part 1,...,146.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Way… One Fate… One Hero.,Harry Potter and the Deathly Hallows: Part 1,False,7.8,13113.0,NaN,NaN
1,False,/kxklJL1v8MYEU5xdU6W5VvmBwVz.jpg,"{'id': 89137, 'name': 'How to Train Your Drago...",165000000.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",http://www.howtotrainyourdragon.com/,10191.0,tt0892769,en,How to Train Your Dragon,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One adventure will change two worlds,How to Train Your Dragon,False,7.8,9190.0,NaN,NaN
2,False,/hCVQPB77eXBGh9abVdrWtKpKgkN.jpg,"{'id': 131292, 'name': 'Iron Man Collection', ...",200000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/iron-man-2,10138.0,tt1228705,en,Iron Man 2,...,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"It's not the armor that makes the hero, but th...",Iron Man 2,False,6.8,14726.0,NaN,NaN
3,False,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.9,12704.0,NaN,NaN
4,False,/s3TBrRGB1iav7gFOCNx3H31MoES.jpg,None,160000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",http://inceptionmovie.warnerbros.com/,27205.0,tt1375666,en,Inception,...,148.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Your mind is the scene of the crime.,Inception,False,8.3,26602.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,None,None,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...",http://www.the2escobars.com,53190.0,tt2700330,en,The Two Escobars,...,100.0,"[{'iso_639_1': 'es', 'name': 'Español'}]",Released,Sometimes it really is a matter of life and de...,The Two Escobars,False,7.5,27.0,NaN,NaN
996,False,None,None,0.0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",http://www.nightbeasts.com/Home.html,248670.0,tt1513835,en,Nightbeasts,...,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,They come out at night. . .,Nightbeasts,False,3.5,4.0,NaN,NaN
997,False,None,None,0.0,"[{'id': 35, 'name': 'Comedy'}]",,43814.0,tt1103255,en,Something Like A Business,...,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's hard out there for a pimp,Something Like A Business,False,3.5,6.0,NaN,NaN
998,False,None,None,0.0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",,39050.0,tt1616512,en,Meadowoods,...,88.0,[],Released,"In a sleepy and uneventful small town, three c...",Meadowoods,False,4.9,7.0,NaN,NaN


In [5]:
# Telling pandas to allow me to see all rows and columns without summarizing
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [14]:
df_nulls = df.isna()['release_date']
df_nulls = df_nulls[lambda x: x == True]
df_nulls
# should get
# 182    True
# 864    True
# need to delete those rows becase they aren't allowing me to view what I want to view

Series([], Name: release_date, dtype: bool)

In [8]:
df = df.drop([182,864])

In [10]:
df['release_date']

0      2010-10-17
1      2010-03-10
2      2010-04-28
3      1995-10-30
4      2010-07-15
5      2010-02-01
6      2009-12-10
7      2010-06-16
8      2010-07-08
9      2010-10-28
10     1999-10-30
11     2010-03-03
12     2010-08-03
13     2010-11-24
14     2010-10-21
15     2010-06-23
16     2010-01-11
17     2010-10-08
18     2004-07-23
19     2010-02-14
20     2002-06-14
21     2010-06-15
22     2010-12-08
23     2010-08-04
24     2010-06-09
25     2010-09-06
26     1995-04-07
27     2010-06-30
28     2010-07-03
29     2010-10-01
30     2010-05-22
31     1995-06-30
32     1998-07-01
33     2007-08-03
34     2010-04-01
35     2010-04-23
36     2010-05-12
37     2010-03-26
38     2010-05-16
39     2010-09-16
40     2010-07-13
41     2010-08-12
42     2010-06-24
43     1959-03-19
44     2010-11-23
45     2010-02-11
46     2010-11-04
47     2010-12-03
48     2010-08-13
49     2010-12-03
50     2010-02-15
51     2010-08-06
52     2010-09-15
53     2010-06-18
54     2010-12-10
55     201

In [16]:
df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,status_code,status_message
0,False,/fXRXpzxUApE3OuXhIqsdavQjCVa.jpg,"{'id': 1241, 'name': 'Harry Potter Collection'...",250000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",https://www.warnerbros.com/movies/harry-potter...,12444.0,tt0926084,en,Harry Potter and the Deathly Hallows: Part 1,"Harry, Ron and Hermione walk away from their l...",34.068,/iGoXIpQb7Pot00EEdwpwPajheZ5.jpg,"[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010-10-17,9.543059e+08,146.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Way… One Fate… One Hero.,Harry Potter and the Deathly Hallows: Part 1,False,7.8,13113.0,NaN,NaN
1,False,/kxklJL1v8MYEU5xdU6W5VvmBwVz.jpg,"{'id': 89137, 'name': 'How to Train Your Drago...",165000000.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",http://www.howtotrainyourdragon.com/,10191.0,tt0892769,en,How to Train Your Dragon,As the son of a Viking leader on the cusp of m...,26.577,/ygGmAO60t8GyqUo9xYeYxSZAR3b.jpg,"[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-03-10,4.948788e+08,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One adventure will change two worlds,How to Train Your Dragon,False,7.8,9190.0,NaN,NaN
2,False,/hCVQPB77eXBGh9abVdrWtKpKgkN.jpg,"{'id': 131292, 'name': 'Iron Man Collection', ...",200000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/iron-man-2,10138.0,tt1228705,en,Iron Man 2,With the world now aware of his dual life as t...,35.234,/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-04-28,6.239333e+08,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"It's not the armor that makes the hero, but th...",Iron Man 2,False,6.8,14726.0,NaN,NaN
3,False,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",38.797,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,3.735540e+08,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.9,12704.0,NaN,NaN
4,False,/s3TBrRGB1iav7gFOCNx3H31MoES.jpg,None,160000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",http://inceptionmovie.warnerbros.com/,27205.0,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",72.244,/9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010-07-15,8.255328e+08,148.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Your mind is the scene of the crime.,Inception,False,8.3,26602.0,NaN,NaN
5,False,/sEyNWq9985lyUrwayWnvicT4FHA.jpg,"{'id': 179919, 'name': 'Percy Jackson Collecti...",95000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",https://www.percyjacksonthemovie.com,32657.0,tt0814255,en,Percy Jackson & the Olympians: The Lightning T...,"Accident prone teenager, Percy discovers he's ...",26.065,/brzpTyZ5bnM7s53C1KSk1TmrMO6.jpg,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-02-01,2.264972e+08,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Worlds Collide,Percy Jackson & the Olympians: The Lightning T...,False,6.2,5274.0,NaN,NaN
6,False,/eS8rJ1KzRNBewx9MduiSHM4kr7S.jpg,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000.0,"[{'id': 28, 'name': '

In [35]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [75]:
genres = []
for item in df['genres']:
    for genre in item:
        genres.append(genre['id'])
genres

[12,
 14,
 14,
 12,
 16,
 10751,
 12,
 28,
 878,
 16,
 35,
 10751,
 28,
 878,
 12,
 12,
 14,
 10751,
 28,
 12,
 14,
 878,
 16,
 10751,
 35,
 16,
 10751,
 16,
 28,
 35,
 10751,
 878,
 16,
 35,
 10751,
 10751,
 14,
 12,
 53,
 12,
 28,
 16,
 10751,
 27,
 80,
 12,
 14,
 18,
 10749,
 28,
 53,
 878,
 10402,
 10749,
 28,
 18,
 53,
 18,
 53,
 9648,
 28,
 18,
 9648,
 53,
 28,
 35,
 12,
 10751,
 14,
 18,
 10749,
 53,
 28,
 12,
 35,
 80,
 18,
 36,
 28,
 35,
 80,
 53,
 28,
 12,
 14,
 28,
 878,
 12,
 53,
 18,
 28,
 53,
 18,
 36,
 28,
 53,
 878,
 12,
 28,
 18,
 9648,
 53,
 12,
 14,
 28,
 35,
 10749,
 28,
 12,
 18,
 35,
 14,
 10751,
 35,
 12,
 14,
 16,
 10751,
 35,
 14,
 12,
 28,
 28,
 35,
 10749,
 10402,
 14,
 35,
 35,
 10402,
 10749,
 80,
 18,
 28,
 53,
 27,
 28,
 53,
 18,
 53,
 27,
 16,
 12,
 10751,
 14,
 10749,
 18,
 12,
 28,
 18,
 28,
 35,
 80,
 80,
 18,
 53,
 28,
 37,
 18,
 14,
 53,
 12,
 28,
 878,
 12,
 14,
 28,
 16,
 10751,
 12,
 9648,
 14,
 18,
 18,
 18,
 12,
 37,
 53,
 80,
 27,
 28,
 80,
 3

In [64]:
profit_by_genre = df.loc[:,['id','budget', 'revenue', 'genres']]
profit_by_genre

,id,budget,revenue,genres
0,12444.0,250000000.0,9.543059e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
1,10191.0,165000000.0,4.948788e+08,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na..."
2,10138.0,200000000.0,6.239333e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."
3,862.0,30000000.0,3.735540e+08,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
4,27205.0,160000000.0,8.255328e+08,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na..."
5,32657.0,95000000.0,2.264972e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
6,19995.0,237000000.0,2.787965e+09,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
7,10193.0,200000000.0,1.066970e+09,"[{'id': 16, 'name': 'Animation'}, {'id': 10751..."
8,20352.0,69000000.0,5.435140e+08,"[{'id': 16, 'name': 'Animation'}, {'id': 10751..."
9,38055.0,130000000.0,3.218858e+08,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '..."


In [67]:
q = """SELECT
        id, budget, revenue, genres
     FROM
        profit_by_genre
     WHERE budget > 0 AND revenue > 0
     """
results = pysqldf(q)

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 3 - probably unsupported type.
[SQL: INSERT INTO profit_by_genre (id, budget, revenue, genres) VALUES (?, ?, ?, ?)]
[parameters: ((12444.0, 250000000.0, 954305868.0, [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}]), (10191.0, 165000000.0, 494878759.0, [{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 10751, 'name': 'Family'}]), (10138.0, 200000000.0, 623933331.0, [{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]), (862.0, 30000000.0, 373554033.0, [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]), (27205.0, 160000000.0, 825532764.0, [{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}]), (32657.0, 95000000.0, 226497209.0, [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'na

In [66]:
results

,id,budget,revenue
0,12444.0,250000000.0,9.543059e+08
1,10191.0,165000000.0,4.948788e+08
2,10138.0,200000000.0,6.239333e+08
3,862.0,30000000.0,3.735540e+08
4,27205.0,160000000.0,8.255328e+08
5,32657.0,95000000.0,2.264972e+08
6,19995.0,237000000.0,2.787965e+09
7,10193.0,200000000.0,1.066970e+09
8,20352.0,69000000.0,5.435140e+08
9,38055.0,130000000.0,3.218858e+08
